# Extra Examples - Merging

Heres a dataset dumped directly from a database, so we need to stitch it together ourselves.
https://www.kaggle.com/uciml/restaurant-data-with-consumer-ratings

The dataset comes with a README file that outlines where everything comes from which might help.

Lets try to:

1. Merge all restaurant data
2. Merge all user data
3. Merge restaurant data and user data together using user ratings
4. Realise that we've merged too much, and merge user ratings + user profile + geoplaces
5. Use some groupby power and determine the top five restaurants in the dataset

In [1]:
import pandas as pd
import os

files = [f for f in os.listdir() if f.endswith(".csv")]
print(files)

['chefmozaccepts.csv', 'chefmozcuisine.csv', 'chefmozhours4.csv', 'chefmozparking.csv', 'geoplaces2.csv', 'rating_final.csv', 'usercuisine.csv', 'userpayment.csv', 'userprofile.csv']


## Merging restaurant data

In [9]:
df_rest = None
for f in files:
    if f.startswith("chef") or f == "geoplaces2.csv":
        df = pd.read_csv(f)
        if df_rest is None:
            df_rest = df
        else:
            df_rest = df_rest.merge(df, on="placeID")
df_rest.head()

placeID Rpayment Rcuisine         hours                  days parking_lot  \
0   135109     cash  Italian  08:00-21:00;  Mon;Tue;Wed;Thu;Fri;        none   
1   135109     cash  Italian  08:00-21:00;                  Sat;        none   
2   135109     cash  Italian  08:00-21:00;                  Sun;        none   
3   135106     cash  Mexican  18:00-23:30;  Mon;Tue;Wed;Thu;Fri;        none   
4   135106     cash  Mexican  18:00-23:30;                  Sat;        none   

    latitude   longitude                                     the_geom_meter  \
0  18.921785  -99.235350  0101000020957F0000A6BF695F136F5AC1DADF87B20556...   
1  18.921785  -99.235350  0101000020957F0000A6BF695F136F5AC1DADF87B20556...   
2  18.921785  -99.235350  0101000020957F0000A6BF695F136F5AC1DADF87B20556...   
3  22.149709 -100.976093  0101000020957F0000649D6F21634858C119AE9BF528A3...   
4  22.149709 -100.976093  0101000020957F0000649D6F21634858C119AE9BF528A3...   

                         name  ...    alcohol   smoking_area dress_code  \
0                   Paniroles  ...  Wine-Beer  not permitted   informal   
1                   Paniroles  ...  Wine-Beer  not permitted   informal   
2                   Paniroles  ...  Wine-Beer  not permitted   informal   
3  El Rinc�n de San Francisco  ...  Wine-Beer    only at bar   informal   
4  El Rinc�n de San Francisco  ...  Wine-Beer    only at bar   informal   

      accessibility   price url Rambience franchise    area other_services  
0  no_accessibility  medium   ?     quiet         f  closed       Internet  
1  no_accessibility  medium   ?     quiet         f  closed       Internet  
2  no_accessibility  medium   ?     quiet         f  closed       Internet  
3         partially  medium   ?  familiar         f    open           none  
4         partially  medium   ?  familiar         f    open           none  

[5 rows x 26 columns]

## Merging User data

In [4]:
df_user = None
for f in files:
    if f.startswith("user"):
        df = pd.read_csv(f)
        if df_user is None:
            df_user = df
        else:
            df_user = df_user.merge(df, on="userID")
df_user.head()

userID  Rcuisine          Upayment   latitude   longitude smoker  \
0  U1001  American              cash  22.139997 -100.978803  false   
1  U1002   Mexican              cash  22.150087 -100.983325  false   
2  U1003   Mexican              cash  22.119847 -100.946527  false   
3  U1004    Bakery              cash  18.867000  -99.183000  false   
4  U1004    Bakery  bank_debit_cards  18.867000  -99.183000  false   

      drink_level dress_preference ambience transport  ...        hijos  \
0      abstemious         informal   family   on foot  ...  independent   
1      abstemious         informal   family    public  ...  independent   
2  social drinker           formal   family    public  ...  independent   
3      abstemious         informal   family    public  ...  independent   
4      abstemious         informal   family    public  ...  independent   

  birth_year    interest          personality  religion      activity  color  \
0       1989     variety    thrifty-protector      none       student  black   
1       1990  technology  hunter-ostentatious  Catholic       student    red   
2       1989        none          hard-worker  Catholic       student   blue   
3       1940     variety          hard-worker      none  professional  green   
4       1940     variety          hard-worker      none  professional  green   

  weight  budget height  
0     69  medium   1.77  
1     40     low   1.87  
2     60     low   1.69  
3     44  medium   1.53  
4     44  medium   1.53  

[5 rows x 21 columns]

## Merging User ratings as well

In [8]:
df_ratings = pd.read_csv("rating_final.csv")
df_ratings = df_ratings.merge(df_user, on="userID")
df_ratings = df_ratings.merge(df_rest, on="placeID")

df_ratings

userID  placeID  rating  food_rating  service_rating Rcuisine_x  \
0      U1077   135085       2            2               2    Mexican   
1      U1077   135085       2            2               2    Mexican   
2      U1077   135085       2            2               2    Mexican   
3      U1077   135085       2            2               2    Mexican   
4      U1077   135085       2            2               2    Mexican   
...      ...      ...     ...          ...             ...        ...   
26404  U1086   132937       2            0               1    Italian   
26405  U1086   132937       2            0               1    Italian   
26406  U1086   132937       2            0               1    Italian   
26407  U1086   132937       2            0               1    Italian   
26408  U1086   132937       2            0               1    Italian   

               Upayment   latitude  longitude smoker  ... activity color  \
0                  VISA  22.156469 -100.98554  false  ...  student  blue   
1                  VISA  22.156469 -100.98554  false  ...  student  blue   
2                  VISA  22.156469 -100.98554  false  ...  student  blue   
3                  cash  22.156469 -100.98554  false  ...  student  blue   
4                  cash  22.156469 -100.98554  false  ...  student  blue   
...                 ...        ...        ...    ...  ...      ...   ...   
26404  bank_debit_cards  22.157281 -100.98444  false  ...  student   red   
26405  bank_debit_cards  22.157281 -100.98444  false  ...  student   red   
26406              cash  22.157281 -100.98444  false  ...  student   red   
26407              cash  22.157281 -100.98444  false  ...  student   red   
26408              cash  22.157281 -100.98444  false  ...  student   red   

      weight  budget height Rpayment  Rcuisine_y         hours  \
0         65  medium   1.71     cash   Fast_Food  00:00-00:00;   
1         65  medium   1.71     cash   Fast_Food  00:00-00:00;   
2         65  medium   1.71     cash   Fast_Food  00:00-00:00;   
3         65  medium   1.71     cash   Fast_Food  00:00-00:00;   
4         65  medium   1.71     cash   Fast_Food  00:00-00:00;   
...      ...     ...    ...      ...         ...           ...   
26404     49  medium   1.59     cash         Bar  21:00-02:00;   
26405     49  medium   1.59     cash         Bar  16:00-23:30;   
26406     49  medium   1.59     cash         Bar  21:00-02:00;   
26407     49  medium   1.59     cash         Bar  21:00-02:00;   
26408     49  medium   1.59     cash         Bar  16:00-23:30;   

                       days parking_lot  
0      Mon;Tue;Wed;Thu;Fri;      public  
1                      Sat;      public  
2                      Sun;      public  
3      Mon;Tue;Wed;Thu;Fri;      public  
4                      Sat;      public  
...                     ...         ...  
26404                  Sat;        none  
26405                  Sun;        none  
26406  Mon;Tue;Wed;Thu;Fri;        none  
26407                  Sat;        none  
26408                  Sun;        none  

[26409 rows x 30 columns]

In [10]:
# So sure we've merged it all together now, but what if we didnt want this? 
# What if we just want user ratings and the place? All this duplicate info
# for parking, opening hours, etc, is useless.

## Merge Subsets

In [31]:
df_u = pd.read_csv("userprofile.csv", index_col="userID")
df_r = pd.read_csv("geoplaces2.csv", index_col="placeID")
df_rate = pd.read_csv("rating_final.csv").merge(df_u, on="userID").merge(df_r, on="placeID")

df_rate

userID  placeID  rating  food_rating  service_rating  latitude_x  \
0     U1077   135085       2            2               2   22.156469   
1     U1108   135085       1            2               1   22.143524   
2     U1081   135085       1            2               1   22.207749   
3     U1001   135085       0            1               1   22.139997   
4     U1056   135085       2            2               2   22.168997   
...     ...      ...     ...          ...             ...         ...   
1156  U1006   132922       2            1               2   22.150000   
1157  U1003   132937       2            2               1   22.119847   
1158  U1027   132937       1            1               1   22.165150   
1159  U1029   132937       1            1               1   22.151796   
1160  U1086   132937       2            0               1   22.157281   

      longitude_x smoker     drink_level dress_preference  ...  \
0     -100.985540  false  social drinker          elegant  ...   
1     -100.987562  false      abstemious         informal  ...   
2     -100.942383  false  casual drinker         informal  ...   
3     -100.978803  false      abstemious         informal  ...   
4     -100.974376  false  social drinker         informal  ...   
...           ...    ...             ...              ...  ...   
1156  -100.983000   true  social drinker    no preference  ...   
1157  -100.946527  false  social drinker           formal  ...   
1158  -100.987015   true  social drinker    no preference  ...   
1159  -100.989075   true  casual drinker           formal  ...   
1160  -100.984440  false  social drinker    no preference  ...   

                alcohol   smoking_area dress_code     accessibility   price  \
0     No_Alcohol_Served  not permitted   informal  no_accessibility  medium   
1     No_Alcohol_Served  not permitted   informal  no_accessibility  medium   
2     No_Alcohol_Served  not permitted   informal  no_accessibility  medium   
3     No_Alcohol_Served  not permitted   informal  no_accessibility  medium   
4     No_Alcohol_Served  not permitted   informal  no_accessibility  medium   
...                 ...            ...        ...               ...     ...   
1156  No_Alcohol_Served      permitted     formal        completely  medium   
1157           Full_Bar        section   informal        completely     low   
1158           Full_Bar        section   informal        completely     low   
1159           Full_Bar        section   informal        completely     low   
1160           Full_Bar        section   informal        completely     low   

     url Rambience franchise    area other_services  
0      ?  familiar         f  closed           none  
1      ?  familiar         f  closed           none  
2      ?  familiar         f  closed           none  
3      ?  familiar         f  closed           none  
4      ?  familiar         f  closed           none  
...   ..       ...       ...     ...            ...  
1156   ?  familiar         f  closed           none  
1157   ?     quiet         f  closed        variety  
1158   ?     quiet         f  closed        variety  
1159   ?     quiet         f  closed        variety  
1160   ?     quiet         f  closed        variety  

[1161 rows x 43 columns]

## Top 5 restaurants based off rating

Note to answer this we didn't actually need the user profile data. But we might use it to remove votes from users that don't satisfy criteria (for example, we might want to make sure the user has been to multiple restaurants, or is a certain age, or doesnt have suspicious voting trends - aka giving everyone a one).

In [40]:
df_small = df_rate[["placeID", "name", "city", "country", "rating"]]
df_small = df_small.set_index("placeID")
top5 = df_small.groupby("placeID").rating.mean().sort_values(ascending=False).index[:5]

df_r.loc[top5, ["name", "address", "city", "country"]]

name  \
placeID                               
132955                    emilianos   
135034   Michiko Restaurant Japones   
134986     Restaurant Las Mananitas   
132922         cafe punta del cielo   
132755         La Estrella de Dimas   

                                             address             city country  
placeID                                                                        
132955                           venustiano carranza   san luis potos       ?  
135034   Cordillera de Los Alpes 160 Lomas 2 Seccion  San Luis Potosi  Mexico  
134986                           Ricardo Linares 107       Cuernavaca  Mexico  
132922                                             ?                ?       ?  
132755                           Av. de los Pintores  San Luis Potosi  Mexico